In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

In [ ]:
def get_agent_list(driver, url, DELAY=10):
    driver.get(url)
    
    try:
        WebDriverWait(driver, DELAY).until(EC.presence_of_element_located((By.CLASS_NAME, 'row.mt-3')))
    except TimeoutException:
        pass
    
    time.sleep(DELAY)

    agent_list = []

    for idx, info in enumerate(driver.find_elements(By.CLASS_NAME, "col-md-6.mt-4")):
        try:
            name =    f"/html/body/div[5]/div[3]/div[2]/div/div/div[2]/div[{idx+1}]/div[1]/div[1]/div[2]/h5/a/span"
            cea =     f"/html/body/div[5]/div[3]/div[2]/div/div/div[2]/div[{idx+1}]/div[1]/div[1]/div[2]/div[2]/div[1]"
            agency =  f"/html/body/div[5]/div[3]/div[2]/div/div/div[2]/div[{idx+1}]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]"
            contact = f"/html/body/div[5]/div[3]/div[2]/div/div/div[2]/div[{idx+1}]/div[1]/div[2]/a[1]"

            agent_list.append({
                "name":    info.find_element(By.XPATH, name).text,
                "cea":     info.find_element(By.XPATH, cea).text.replace("CEA: ",""),
                "agency":  info.find_element(By.XPATH, agency).text,
                "contact": info.find_element(By.XPATH, contact).get_attribute("data-clipboard-text"),
            })
        except Exception as e:
            print(idx, e)

    return agent_list

In [ ]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(executable_path="./chromedriver_mac64/chromedriver"),
                          options=chrome_options)

In [ ]:
all_agents = []

for page_no in range(1,258):
    URL = f"https://www.srx.com.sg/find-agents/search?page={page_no}&maxResults=30"
    print(page_no, URL)
    agents = get_agent_list(driver, URL)
    all_agents.extend(agents)

In [ ]:
import pandas as pd

agents_df = pd.DataFrame(all_agents)
agents_df

In [ ]:
agents_df.to_csv("sg_srx_all_real_estate_agents.csv", index=False)

## PropertyGuru

In [40]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(executable_path="./chromedriver_mac64/chromedriver"),
                          options=chrome_options)

DELAY = 5

In [ ]:
agent_link_list = []

for page_no in range(1, 5):
    url = f"https://www.propertyguru.com.sg/property-agent-directory/search/{page_no}?locale=en&order=asc&property_specialist_type_code=HDB&sort=firstname"

    driver.get(url)

    try:
        WebDriverWait(driver, DELAY).until(EC.presence_of_element_located((By.CLASS_NAME, 'agent-list')))
    except TimeoutException:
        pass

    time.sleep(DELAY)

    for agent_card in driver.find_elements(By.CLASS_NAME, "agent-card"):
        agent_info = eval(agent_card.get_attribute('data-ec-impression'))['productData']

        agent_links = {"name": agent_info['name'],
                    "agency": agent_info['brand'],
                    "link": agent_card.find_element(By.TAG_NAME, "a").get_attribute("href")}

        agent_link_list.append(agent_links)

In [ ]:
import pandas as pd
pd.DataFrame(agent_link_list)